## This notebook chooses the best model and based on its predictions on validation set, it calculates a SIGMA MAD validation metric

In [7]:
%load_ext autoreload
%autoreload 2
import os 
import sys
import time
import h2o

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.modeling.functions_autoML import gen_predictions

In [19]:
pd.set_option("display.max_rows", 6)

In [36]:
df = pd.read_pickle("../../data/structured/df.pkl")
#IA Filter
df = df.loc[df.type_bool == True]

#### Getting h2o best result and making a prediction to list

In [12]:
h2o.init(ip="localhost", port=54323)
best_model = h2o.import_mojo('../../models/mojo_40_ensemble/StackedEnsemble_AllModels_AutoML_20201222_113425.zip')

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1608689537668_2

No model summary for this model

ModelMetricsRegressionGeneric: generic
** Reported on train data. **

MSE: 0.002250204251768219
RMSE: 0.04743631785634525
MAE: 0.035373368298983764
RMSLE: 0.026542703609754942
Mean Residual Deviance: 0.002250204251768219

ModelMetricsRegressionGeneric: generic
** Reported on cross-validation data. **

MSE: 0.00761592126008453
RMSE: 0.08726924578615614
MAE: 0.06693126534743787
RMSLE: 0.050119043057072946
Mean Residual Deviance: 0.00761592126008453



## Predicting values based on data used to validation

In [51]:
preds, len_train, len_validation = gen_predictions(df, best_model)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
generic prediction progress: |████████████████████████████████████████████| 100%

 2550 train/test objects  
 and 2528 validation objects


## Creates new column on df. It will be used for SIGMA MAD calculus.

In [52]:
df_sigma_mad = df.iloc[-len_validation:]
df_sigma_mad['zphot'] = preds.as_data_frame()['predict'].tolist()
df_sigma_mad

/home/fmoliveira/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC,zphot
ID,,,,,,,,,,,,,,,,,,,,,
SN510924,"[[8.031000000002678, 5.216, 6.285], [28.879000...","[[13.93699999999808, 2.199, 7.08], [44.8870000...","[[0.0, 2.088, 2.24], [13.042999999997846, 1.82...","[[13.953000000001339, 1.99, 5.422], [36.940999...","[0.0, 0.7661717171717413, 1.5323434343434825, ...","([8.031000000002678, 8.70594949495217, 9.38089...","([13.93699999999808, 14.552414141412289, 15.16...","([0.0, 0.7561111111111435, 1.512222222222287, ...","([13.953000000001339, 14.578232323233673, 15.2...","[10.589545779483814, 8.708973184516886, 7.2444...",...,-39.555982,-16.293731,47.963569,-45.448702,-27.989244,-6.281132,Ia,True,0.85640,0.868346
SN511436,"[[1.9689999999973224, 1.07, 3.215], [10.089999...","[[0.01600000000325963, 51.65, 9.372], [2.0, 49...","[[0.0, 35.65, 7.882], [1.9839999999967404, 32....","[[0.9610000000029686, 54.02, 5.43], [9.9409999...","[0.0, 0.8387777777777478, 1.6775555555554955, ...","([1.9689999999973224, 2.6458080808054114, 3.32...","([0.01600000000325963, 0.8545353535386073, 1.6...","([0.0, 0.7076262626262656, 1.4152525252525312,...","([0.9610000000029686, 1.7900707070736157, 2.61...","[9.27014418909458, 8.255517290037698, 7.745205...",...,23.134718,128.783691,-8.731676,12.435225,17.675975,31.130664,Ia,True,0.58313,0.542263
SN511804,"[[2.989000000001397, 3.118, 2.585], [13.055000...","[[0.01599999999598367, 17.51, 3.166], [3.03499...","[[0.0, 1.033, 2.13], [4.082000000002154, 5.482...","[[0.03899999999703141, 19.44, 2.511], [3.06700...","[0.0, 1.1714444444444256, 2.3428888888888513, ...","([2.989000000001397, 4.129939393940805, 5.2708...","([0.01599999999598367, 1.1872828282788337, 2.3...","([0.0, 1.1302525252524929, 2.2605050505049857,...","([0.03899999999703141, 1.208666666663758, 2.37...","[5.178899315805631, 6.259860318410908, 7.21610...",...,18.961628,22.237930,22.428448,16.678827,0.003466,13.356164,Ia,True,0.99576,0.985524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN99438,"[[0.0, 2.33, 2.55], [7.894000000000233, 3.805,...","[[0.030999999995401595, 0.006543, 3.683], [7.9...","[[0.014999999999417923, 0.4446, 1.3], [14.0039...","[[0.06199999999807915, 1.497, 4.486], [7.97199...","[0.0, 1.7255757575757058, 3.4511515151514116, ...","([0.0, 1.6948383838383538, 3.3896767676767077,...","([0.030999999995401595, 1.75597979797521, 3.48...","([0.014999999999417923, 1.7298888888882826, 3....","([0.06199999999807915, 1.7869494949475417, 3.5...","[6.789900493579756, 7.110462975693107, 7.57937...",...,-34.417710,32.878025,-33.942007,34.613412,61.324876,-51.210969,Ia,True,0.75966,0.732788
SN99457,"[[0.0, 10.06, 4.768], [4.075000000004366, 4.39...","[[3.0429999999978463, 0.9564, 1.984], [4.10599...","[[3.004000000000815, 1.456, 1.709], [4.0900000...","[[4.137000000002445, 9.726, 4.49], [12.0469999...","[0.0, 1.7667373737373662, 3.5334747474747323, ...","([0.0, 1.7360000000000142, 3.4720000000000284,...","([3.0429999999978463, 4.778717171715048, 6.514...","([3.004000000000815, 4.729858585859406, 6.4557...","([4.137000000002445, 5.861949494951907, 7.5868...","[8.252504299669107, 11.125114322870362, 13.374...",...,9.078515,-15.904951,34.732243,-14.319036,-22.934570,-9.153751,Ia,True,0.80886,0.883200
SN99946,"[[11.961000000002969, 1.526, 5.683], [13.05500...","[[0.01599999999598367, 4.95, 3.147], [3.034999...","[[0.0, 0.4826, 2.129], [2.995999999999185, 0.7...","[[0.03899999999703141, 1.045, 2.483], [12.0389...","[0.0, 1.3626161616161374, 2.725232323232275, 4...","([11.961000000002969, 13.202242424245336, 14.4...","([0.01599999999598367, 1.3782121212081455, 2.7...","([0.0, 1.362181818181816, 2.724363636363632, 4...","([0.03899999999703141, 1.4012222222192594, 2.7...","[3.7693816011787566, 5.06053615747151, 6.78000...",...,183.911735,62.540835,28.895779,-271.089840,54.766397,39.959973,Ia,True,0.40625,0.413554


In [23]:
#pd.concat([X.loc[df_sigma_mad.index],df_sigma_mad['zphot']], axis = 1).to_csv('features_target.csv')
#pd.concat([X.loc[df_sigma_mad.index],df_sigma_mad['zphot']], axis = 1).to_pickle('features_target.pkl')

## Deltas and Sigmas calculus

In [71]:
get_delta_z = lambda zphot, zspec : (zphot - zspec)/(1 + zspec)

In [87]:
delta_z = get_delta_z(df_sigma_mad.REDSHIFT_SPEC.values, df_sigma_mad.zphot.values)
delta_z

array([-0.00854058, -0.09044255, -0.0079693 , ...,  0.18464257,
        0.01135406, -0.26343139])

In [86]:
median_delta_z = np.median(delta_z)
median_delta_z

0.0007232058109612594

In [85]:
MAD = np.median(np.abs(delta_z - median_delta_z))
MAD

0.1213389099225449

In [84]:
sigma_MAD = 1.4826*MAD
sigma_MAD

0.17989706785116505

In [95]:
n_fraction = sum(np.abs(delta_z) > 0.05)/len(delta_z)
n_fraction

0.7730496453900709